In [2]:
import random
from ase import Atoms
from ase.io import Trajectory
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import csv
import os

import torch
import torch.nn as nn
from torch.autograd.functional import jacobian

random.seed(88)

In [7]:
# Example: Define a simple neural network model
class ForceModel(nn.Module):
    def __init__(self):
        super(ForceModel, self).__init__()
        self.fc1 = nn.Linear(3, 10)  # Input is 3D position
        self.fc2 = nn.Linear(10, 3)  # Output is 3D force (F_x, F_y, F_z)

    def forward(self, pos):
        x = torch.relu(self.fc1(pos))
        return x, self.fc2(x)  # Output is a force vector of shape (3,)

# Create an instance of the model
model = ForceModel()

# Example input: position tensor (batch size = 1, 3 components)
pos = torch.tensor([[1.0, 2.0, 3.0]], requires_grad=True)

# Define the force_func to use the model's forward pass
def force_func(pos):
    return model(pos)[1]  # Model output is the force corresponding to the position

# Compute the Jacobian of the force with respect to position
J = jacobian(force_func, pos)  # Jacobian shape will be [1, 3, 3]

print("Jacobian of force with respect to position:\n", J)

Jacobian of force with respect to position:
 tensor([[[[ 0.0658, -0.1234, -0.0687]],

         [[-0.1036,  0.1432,  0.2423]],

         [[-0.0864,  0.1067,  0.0709]]]])


In [14]:
# Example data
N = 5  # Number of rows
pos = torch.randn(N, 3, requires_grad=True)  # Position matrix [N, 3]

# Define a simple function to compute forces as a function of pos
def compute_forces(pos):
    # Example: forces derived from a simple linear transformation
    return torch.matmul(pos, torch.randn(3, 3, requires_grad=True))  # [N, 3]

# Compute forces based on pos
F = compute_forces(pos)  # F depends on pos, so it has a grad_fn

# Index of the row for which to compute the partial Jacobian
i = 2  # Choose the third row (0-based indexing)

# Select the row of F we're interested in
F_i = F[i]  # Shape: [3]

# Compute gradients of F_i w.r.t pos[i]
J_i = []
for k in range(3):  # Iterate over components of F_i
    grad_outputs = torch.zeros_like(F)  # Shape: [N, 3]
    grad_outputs[i, k] = 1.0  # Focus on F[i, k]
    grad_pos = torch.autograd.grad(F, pos, grad_outputs=grad_outputs, retain_graph=True)[0]
    J_i.append(grad_pos[i])  # Only the gradient w.r.t. pos[i]

# Stack to form the 3x3 Jacobian
J_i = torch.stack(J_i)  # Shape: [3, 3]

print("Jacobian of F[{}] w.r.t pos[{}]:\n".format(i, i), J_i)


Jacobian of F[2] w.r.t pos[2]:
 tensor([[ 0.0346, -1.0355,  1.0360],
        [-0.6227, -0.2849, -0.5619],
        [-1.9283, -0.2640, -0.5997]])
